# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize #Aditya
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())

train_doc = np.random.permutation(open('Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/Project_data/val.csv').readlines())

batch_size = 5 #experiment with the batch size
img_size = 120
interval = 5
frame_count=30

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(0,frame_count,interval) #Aditya  #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size #Aditya # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            x = len(img_idx)
            y = img_size
            z = img_size
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image_resized = imresize(image, size=(img_size,img_size)) #Aditya

                    # Aditya - pending normalization
                    batch_data[folder,idx,:,:,0] = image_resized[:,:,0]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image_resized[:,:,1]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image_resized[:,:,2]/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
# curr_dt_time = datetime.datetime.now()
# train_path = '/notebooks/storage/Final_data/Collated_training/train'
# val_path = '/notebooks/storage/Final_data/Collated_training/val'
# num_train_sequences = len(train_doc)
# print('# training sequences =', num_train_sequences)
# num_val_sequences = len(val_doc)
# print('# validation sequences =', num_val_sequences)
# num_epochs = # choose the number of epochs
# print ('# epochs =', num_epochs)

curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/Project_data/train'
val_path = 'Project_data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, LSTM, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
#Aditya

NUM_CLASSES = 5

# With Conv3D
input_shape = (len(range(0,frame_count,interval)),img_size,img_size,3)
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', padding='same',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

model.add(Dropout(0.25))

model.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', padding='same'))

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3),
                 activation='relu')) # input shape = (img_rows, img_cols, 1)

model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3),
                 activation='relu', padding='same'))

model.add(Flatten())

model.add(Dense(NUM_CLASSES, activation='softmax'))

print (model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 6, 120, 120, 32)   2624      
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 120, 120, 32)   0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 6, 120, 120, 32)   27680     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 60, 60, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 58, 58, 64)     55360     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 58, 58, 64)     0         
_______________________

In [7]:
# model = Sequential()

# model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),
#     activation='relu', padding='same'), input_shape=input_shape))
# model.add(TimeDistributed(Conv2D(32, (3,3),
#     kernel_initializer="he_normal", activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# model.add(TimeDistributed(Conv2D(64, (3,3),
#     padding='same', activation='relu')))
# model.add(TimeDistributed(Conv2D(64, (3,3),
#     padding='same', activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# model.add(TimeDistributed(Conv2D(128, (3,3),
#     padding='same', activation='relu')))
# model.add(TimeDistributed(Conv2D(128, (3,3),
#     padding='same', activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# model.add(TimeDistributed(Conv2D(256, (3,3),
#     padding='same', activation='relu')))
# model.add(TimeDistributed(Conv2D(256, (3,3),
#     padding='same', activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# model.add(TimeDistributed(Conv2D(512, (3,3),
#     padding='same', activation='relu')))
# model.add(TimeDistributed(Conv2D(512, (3,3),
#     padding='same', activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# model.add(TimeDistributed(Flatten()))

# model.add(Dropout(0.5))
# model.add(LSTM(256, return_sequences=False, dropout=0.5))
# model.add(Dense(NUM_CLASSES, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
learning_rate = 0.01
optimiser = optimizers.Adam(lr=learning_rate,clipvalue=1.0) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 6, 120, 120, 32)   2624      
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 120, 120, 32)   0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 6, 120, 120, 32)   27680     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 60, 60, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 58, 58, 64)     55360     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 58, 58, 64)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 1, 58, 58, 64)     110656    
________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)# write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Source path = Source path =  Project_data/Project_data/train  Project_data/Project_data/val ; batch size = 5
; batch size = 5
Epoch 1/10


C:\Users\adi00\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
C:\Users\adi00\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


133/133 [==============================] - 97s 729ms/step - loss: 12.9066 - categorical_accuracy: 0.1940 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2020-12-1820_21_01.561367/model-00001-12.90656-0.19398-12.57211-0.22000.h5
Epoch 2/10
133/133 [==============================] - 87s 655ms/step - loss: 12.9672 - categorical_accuracy: 0.1955 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2020-12-1820_21_01.561367/model-00002-12.96719-0.19549-12.57211-0.22000.h5
Epoch 3/10
133/133 [==============================] - 88s 663ms/step - loss: 12.9672 - categorical_accuracy: 0.1955 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_init_2020-12-1820_21_01.561367/model-00003-12.96719-0.19549-12.57211-0.22000.h5
Epoch 4/10
127/133 [===========================>..] - ETA: 3s - loss: 12.8691 - categorical_accuracy: 0.2016